# Part 1.1 & 1.2 - Into

In [ ]:
import openai
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
import os

In [ ]:
os.environ

In [ ]:
response = client.chat.competions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course"}]
)

In [ ]:
response.choices[0].message.content

# Part 1.3 - Retrivel and Search

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [ ]:
import minsearch
import json

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
docs_raw

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
q = 'the course has already started, can I still enroll?'

In [ ]:
index.fit(documents)

In [ ]:
boost = {
    'question': 3.0,
    'section': 0.5
}

index.search(
    query=q,
    filter_dict={
        'course': 'data-engineering-zoomcamp'
    },
    boost_dict=boost,
    num_results=5
)

# Part 1.4.1 Generation

- Invoking OpenAI API
- Building the prompt
- Getting the answer

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
q = 'the course has already started, can I still enroll?'

In [ ]:
response  = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [ ]:
response.choices[0].message.content

In [ ]:
prompt_template = """
You're a course yeaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answert, output NONE.


QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [ ]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n'n"

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
print(prompt)

In [ ]:
response  = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

# Part 1.4.2 - Exploring Alternatives to OpenAI

- MistralAI

# Part 1.5 - Cleaned RAG flow

- Cleaning the code we wrote so far
- MAking it modular

In [ ]:
def search(query):
    boost = {
        'question': 3.0,
        'section': 0.5
    }
    
    results = index.search(
        query=query,
        filter_dict={
            'course': 'data-engineering-zoomcamp'
        },
        boost_dict=boost,
        num_results=5
    )

    return results

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course yeaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answert, output NONE.
    
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n'n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response  = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
query = 'how do i run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

In [ ]:
rag(query)

# Part 1.6 - Search with Elasticsearch

```bash
docker run -it \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

```bash
curl http://localhost:9200
```

In [ ]:
documents[0]

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch(
    'http://localhost:9200',
    headers={"Content-Type": "application/vnd.elasticsearch+json;compatible-with=8"}
)

In [ ]:
es_client.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
documents[0]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
query = 'I just disovered the course. Can I still join it?'

In [ ]:
rag(query)